In [8]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.models import load_model, model_from_json

In [9]:
# path for Kaggle kernels
input_path = "./data/aug2/"

In [10]:
#rescale=1./255,

train_datagen = ImageDataGenerator(
    shear_range=40,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    brightness_range=(0.1, 0.9),
    channel_shift_range=150.0,
    fill_mode='reflect',
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    input_path + 'train',
    batch_size=32,
    class_mode='categorical',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    input_path + 'val',
    shuffle=False,
    batch_size=32,
    class_mode='categorical',
    target_size=(224,224))

Found 84 images belonging to 6 classes.
Found 16 images belonging to 6 classes.


In [20]:
#Resnet50
conv_base = ResNet50(include_top=False,weights='imagenet')
#InceptionResnetV2
#conv_base=keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet')
#VGG19
#conv_base=tensorflow.keras.applications.vgg19.VGG19(include_top=True, weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

/home/e43879/miniconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [23]:
drop_rate=0.5
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(drop_rate)(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(drop_rate)(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(drop_rate)(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(drop_rate)(x)
predictions = layers.Dense(6, activation='softmax')(x)
model = Model(conv_base.input, predictions)
#model.summary()
optimizer = tensorflow.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=100,  # added in Kaggle
                              epochs=10,
                              validation_data=validation_generator,
                              validation_steps=50  # added in Kaggle
                             )

Epoch 1/10
100/100 [==============================] - 85s 855ms/step - loss: 1.5050 - acc: 0.4198 - val_loss: 0.9291 - val_acc: 0.6875
Epoch 2/10
 92/100 [==========================>...] - ETA: 4s - loss: 0.4564 - acc: 0.8383

KeyboardInterrupt: 

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.figure(figsize=(5, 5))
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'test_acc'], loc='lower left')
plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss'], loc='upper left')
plt.ylim([0, 2])
plt.tight_layout()

In [ ]:
import os
path=input_path+"val"
files = []
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))
        
validation_img_paths = files
img_list = [Image.open(img_path).convert("RGB") for img_path in validation_img_paths]

validation_batch = np.stack([preprocess_input(np.array(img.resize((224,224)))) for img in img_list])
pred_probs = model.predict(validation_batch)

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(30, 30))
for i, img in enumerate(img_list):
    if i<5:
        ax = axs[0,i]
        ax.axis('off')
        ax.set_title(" MIT {:.0f}% , AM {:.0f}% ,X {:.0f}%,STAN {:.0f}%,CS {:.0f}%,HARV {:.0f}% ".format(
            100*pred_probs[i,0],100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3],100*pred_probs[i,4],100*pred_probs[i,5]))
        ax.imshow(img)
    if i<10 and i>=5:
        ax = axs[1,5-i]
        ax.axis('off')
        ax.set_title("MIT {:.0f}% , AM {:.0f}% ,X {:.0f}%,STAN {:.0f}%,CS {:.2f}%,HARV {:.0f}%".format(
            100*pred_probs[i,0],100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3],100*pred_probs[i,4],100*pred_probs[i,5]))
        ax.imshow(img)
    if i>=10 and i<15:
        ax = axs[2,10-i]
        ax.axis('off')
        ax.set_title("MIT {:.0f}% , AM {:.0f}% ,X {:.0f}%,STAN {:.0f}%,CS {:.2f}%,HARV {:.0f}%".format(
            100*pred_probs[i,0],100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3],100*pred_probs[i,4],100*pred_probs[i,5]))
        ax.imshow(img)
    if i>=15:
        ax = axs[3,15-i]
        ax.axis('off')
        ax.set_title("MIT {:.0f}% , AM {:.0f}% ,X {:.0f}%,STAN {:.0f}%,CS {:.2f}%,HARV {:.0f}%".format(
            100*pred_probs[i,0],100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3],100*pred_probs[i,4],100*pred_probs[i,5]))
        ax.imshow(img)